In [224]:
import numpy as np
import torch
import torch.nn.functional as F
from typing import List, Optional, Tuple, Union
from transformers import PreTrainedModel, AutoModelForSequenceClassification
from transformers import AutoTokenizer

import math

from typing import List, Optional, Tuple, Union
from transformers import BertForSequenceClassification
import transformers
from transformers.modeling_outputs import SequenceClassifierOutput

In [225]:
from modeling_rmt_enc_dec import RMTEncoderDecoderForConditionalGeneration

### Finetune

In [226]:
from transformers import AutoModelForSequenceClassification
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
import sys
sys.path.append('..')
# from modeling_rmt import RMTEncoderForSequenceClassification

In [227]:
import math
import types

from transformers.modeling_outputs import BaseModelOutputWithPastAndCrossAttentions

def encoder_memory_forward(
    self,
    hidden_states: torch.Tensor,
    attention_mask: Optional[torch.FloatTensor] = None,
    head_mask: Optional[torch.FloatTensor] = None,
    encoder_hidden_states: Optional[torch.FloatTensor] = None,
    encoder_attention_mask: Optional[torch.FloatTensor] = None,
    past_key_values: Optional[Tuple[Tuple[torch.FloatTensor]]] = None,
    use_cache: Optional[bool] = None,
    output_attentions: Optional[bool] = False,
    output_hidden_states: Optional[bool] = False,
    return_dict: Optional[bool] = True,
    memory_storage = None
) -> Union[Tuple[torch.Tensor], BaseModelOutputWithPastAndCrossAttentions]:
    '''
    Copy-pasted from BERT encoder
    '''
    all_hidden_states = () if output_hidden_states else None
    all_self_attentions = () if output_attentions else None
    all_cross_attentions = () if output_attentions and self.config.add_cross_attention else None

    next_decoder_cache = () if use_cache else None
    for i, layer_module in enumerate(self.layer):
        if output_hidden_states:
            all_hidden_states = all_hidden_states + (hidden_states,)

        layer_head_mask = head_mask[i] if head_mask is not None else None
        past_key_value = past_key_values[i] if past_key_values is not None else None

        if self.gradient_checkpointing and self.training:

            # TBD: load logger 
            # if use_cache:
            #     logger.warning(
            #         "`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`..."
            #     )
            #     use_cache = False

            def create_custom_forward(module):
                def custom_forward(*inputs):
                    return module(*inputs, past_key_value, output_attentions)

                return custom_forward

            layer_outputs = torch.utils.checkpoint.checkpoint(
                create_custom_forward(layer_module),
                hidden_states,
                attention_mask,
                layer_head_mask,
                encoder_hidden_states,
                encoder_attention_mask,
            )
        else:
            num_mem = memory_storage['num_mem_tokens']
            if (i in memory_storage) and (i > 0):
                layer_memory = memory_storage[i]
                for j, h in enumerate(hidden_states):
                    hidden_states[j][:layer_memory[j].shape[0]] = layer_memory[j]

            # print(f'hidden states shape: {len(hidden_states), hidden_states[0].shape}\n memory storage:{memory_storage.keys()}')
            layer_outputs = layer_module(
                hidden_states,
                attention_mask,
                layer_head_mask,
                encoder_hidden_states,
                encoder_attention_mask,
                past_key_value,
                output_attentions,
            )
            
        hidden_states = layer_outputs[0]
        if i in memory_storage:
            # print(f'replacing ms[i] {memory_storage[i][0][0][:10]}... to {[h[:num_mem] for h in hidden_states][0][0][:10]}')
            memory_storage[i] = [h[:num_mem] for h in hidden_states]

        # print(f'Overrided method message: hidden states shape: {len(hidden_states), hidden_states[0].shape}\n memory storage:{memory_storage}')
        if use_cache:
            next_decoder_cache += (layer_outputs[-1],)
        if output_attentions:
            all_self_attentions = all_self_attentions + (layer_outputs[1],)
            if self.config.add_cross_attention:
                all_cross_attentions = all_cross_attentions + (layer_outputs[2],)

    if output_hidden_states:
        all_hidden_states = all_hidden_states + (hidden_states,)

    if not return_dict:
        return tuple(
            v
            for v in [
                hidden_states,
                next_decoder_cache,
                all_hidden_states,
                all_self_attentions,
                all_cross_attentions,
            ]
            if v is not None
        )
    return BaseModelOutputWithPastAndCrossAttentions(
        last_hidden_state=hidden_states,
        past_key_values=next_decoder_cache,
        hidden_states=all_hidden_states,
        attentions=all_self_attentions,
        cross_attentions=all_cross_attentions,
    )


class RMTEncoderForSequenceClassification():
    '''
    Usage
    
    Way 1: from config
    rmt = RMTEncoderForSequenceClassification(config=config)

    Way 2: from HF model name
    model_name = "bert-base-cased"
    rmt = RMTEncoderForSequenceClassification.from_pretrained(model_name)

    Way 3: from instance of HF model
    model = AutoModelForSequenceClassification("bert-base-cased") 
    rmt = RMTEncoderForSequenceClassification(base_model=model)

    '''
    def __init__(self, config=None, base_model=None, **kwargs):
        if config is not None:
            self.model = AutoModelForSequenceClassification(config, **kwargs)
        
        if base_model is not None:
            self.model = base_model


    def from_pretrained(from_pretrained, **kwargs):
        base_model = AutoModelForSequenceClassification.from_pretrained(from_pretrained, **kwargs)
        rmt = RMTEncoderForSequenceClassification(base_model=base_model)
        return rmt
        

    def set_params(self, 
                backbone_cls=None,
                num_mem_tokens=0, 
                inter_layer_memory=False,
                segment_ordering='regular',
                input_size=None, 
                input_seg_size=None, 
                bptt_depth=-1, 
                drop_empty_segments=True,
                sum_loss=False,
                # special_tokens=None,
                tokenizer=None
                  ):
        
        if input_size is not None:
            self.input_size = input_size
        else:
            self.input_size =  self.base_model.embeddings.position_embeddings.weight.shape[0]
        self.input_seg_size = input_seg_size

        # print('\n\n\n', special_tokens)
        
        self.bptt_depth = bptt_depth
        # self.pad_token_id = special_tokens['pad_token_id']
        # self.cls_token = torch.tensor([special_tokens['cls_token_id']])
        # self.sep_token = torch.tensor([special_tokens['sep_token_id']])
        self.pad_token_id = tokenizer.pad_token_id
        self.cls_token = torch.tensor([tokenizer.cls_token_id])
        self.sep_token = torch.tensor([tokenizer.sep_token_id])
        
        self.num_mem_tokens = num_mem_tokens
        self.segment_ordering = segment_ordering
        self.drop_empty_segments = drop_empty_segments
        self.sum_loss = sum_loss
        self.extend_word_embeddings()

        if inter_layer_memory:
            self.override_encoder_forward()


    def set_memory(self, memory=None):
        if memory is None:
            mem_token_ids = self.mem_token_ids.to(device=self.device)
            memory = self.base_model.embeddings.word_embeddings(mem_token_ids)
        return memory
    
    def extend_word_embeddings(self):
        vocab_size = self.base_model.embeddings.word_embeddings.weight.shape[0]
        extended_vocab_size = vocab_size + self.num_mem_tokens
        self.mem_token_ids = torch.arange(vocab_size, vocab_size + self.num_mem_tokens)
        self.base_model.resize_token_embeddings(extended_vocab_size)
    
    def override_encoder_forward(self):
        self.memory_storage = {'num_mem_tokens': self.num_mem_tokens}
        memory_forward = lambda *args, **kwargs: encoder_memory_forward(*args, **kwargs, memory_storage=self.memory_storage)
        self.base_model.encoder.forward = types.MethodType(memory_forward, self.base_model.encoder)


    def __call__(self, input_ids, **kwargs):
        memory = self.set_memory()
        segmented = self.pad_and_segment(input_ids)
        segmented = list(zip(*segmented))

        if self.segment_ordering in {'regular', 'last_memory_only'}:
            pass
        elif self.segment_ordering == 'reversed':
            segmented == segmented[::-1]
        elif self.segment_ordering == 'bidirectional':
            segmented == segmented + segmented[::-1][1:]
        elif self.segment_ordering == 'repeat_first':
            segmented == segmented + segmented[:1]
        else:
            raise ValueError(f'Unknown segment ordering: {segment_ordering}')

        self.memory_storage = {'num_mem_tokens': self.num_mem_tokens}
        outputs = []
        for seg_num, segment_data in enumerate(segmented):
            input_ids, attention_mask, token_type_ids = segment_data
            if memory.ndim == 2:
                memory = memory.repeat(input_ids.shape[0], 1, 1)
            if (self.bptt_depth > -1) and (len(segmented) - seg_num > self.bptt_depth): 
                memory = memory.detach()

            seg_kwargs = dict(**kwargs)
            if self.drop_empty_segments:
                non_empty_mask = [not torch.equal(input_ids[i], self.empty) for i in range(len(input_ids))]
                if sum(non_empty_mask) == 0:
                    continue
                input_ids = input_ids[non_empty_mask]
                attention_mask = attention_mask[non_empty_mask]
                token_type_ids = token_type_ids[non_empty_mask]
                seg_kwargs['labels'] = seg_kwargs['labels'][non_empty_mask]

                inputs_embeds = self.base_model.embeddings.word_embeddings(input_ids)
                inputs_embeds[:, 1:1+self.num_mem_tokens] = memory[non_empty_mask]
            else:
                inputs_embeds = self.base_model.embeddings.word_embeddings(input_ids)
                inputs_embeds[:, 1:1+self.num_mem_tokens] = memory

            seg_kwargs['inputs_embeds'] = inputs_embeds
            seg_kwargs['attention_mask'] = attention_mask
            seg_kwargs['token_type_ids'] = token_type_ids
            
            out = self.model.forward(**seg_kwargs, output_hidden_states=True)
            outputs.append(out)

            if self.drop_empty_segments:
                memory[non_empty_mask] = out.hidden_states[-1][:, :self.num_mem_tokens]
            else:
                memory = out.hidden_states[-1][:, :self.num_mem_tokens]

        for i, o in enumerate(outputs):
            out[f'loss_{i}'] = o['loss'].mean()

        if self.sum_loss:
            out['loss'] = torch.stack([o['loss'] for o in outputs]).sum(dim=-1)
            
        return out

    def pad_and_segment(self, input_ids):
        
        sequence_len = input_ids.shape[1]
        input_seg_size = self.input_size - self.num_mem_tokens - 3 
        if self.input_seg_size is not None and self.input_seg_size < input_seg_size:
            input_seg_size = self.input_seg_size
            
        n_segments = math.ceil(sequence_len / input_seg_size)

        augmented_inputs = []
        for input in input_ids:
            input = input[input != self.pad_token_id][1:-1]

            seg_sep_inds = [0] + list(range(len(input), 0, -input_seg_size))[::-1] # chunk so that first segment has various size
            input_segments = [input[s:e] for s, e in zip(seg_sep_inds, seg_sep_inds[1:])]

            def pad_add_special_tokens(tensor, seg_size):
                tensor = torch.cat([self.cls_token.to(device=self.device),
                                    self.mem_token_ids.to(device=self.device),
                                    self.sep_token.to(device=self.device),
                                    tensor.to(device=self.device),
                                    self.sep_token.to(device=self.device)])
                pad_size = seg_size - tensor.shape[0]
                if pad_size > 0:
                    tensor = F.pad(tensor, (0, pad_size))
                return tensor

            input_segments = [pad_add_special_tokens(t, self.input_size) for t in input_segments]
            empty = torch.Tensor([]).int()
            self.empty = pad_add_special_tokens(empty, self.input_size)
            empty_segments = [self.empty for i in range(n_segments - len(input_segments))]
            input_segments = empty_segments + input_segments

            augmented_input = torch.cat(input_segments)
            augmented_inputs.append(augmented_input)
            
        augmented_inputs = torch.stack(augmented_inputs)
        attention_mask = torch.ones_like(augmented_inputs)
        attention_mask[augmented_inputs == self.pad_token_id] = 0

        token_type_ids = torch.zeros_like(attention_mask)

        input_segments = torch.chunk(augmented_inputs, n_segments, dim=1)
        attention_mask = torch.chunk(attention_mask, n_segments, dim=1)
        token_type_ids = torch.chunk(token_type_ids, n_segments, dim=1)
    
        return input_segments, attention_mask, token_type_ids


    def to(self, device):
        self.model = self.model.to(device)
        
    
    def cuda(self):
        self.model.cuda()


    def __getattr__(self, attribute):
        return getattr(self.model, attribute)


    def parameters(self, **kwargs):
        return self.model.parameters(**kwargs)

    def named_parameters(self, **kwargs):
        return self.model.named_parameters(**kwargs)

In [228]:
# pretrained_model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5, output_hidden_states=True)

In [5]:
model_name = "google/bert_uncased_L-4_H-256_A-4"
# model_name = "microsoft/deberta-v3-base"
# model_name = "facebook/bart-base"
# model_name = 'google/electra-base-discriminator'

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

/cephfs/home/bulatov/bulatov/hvdenv/lib/python3.8/site-packages/cryptography/hazmat/backends/openssl/x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


In [9]:
num_segments = 2
num_mem_tokens = 10

tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer.model_max_length  = (tokenizer.model_max_length - num_mem_tokens) * num_segments
tokenizer.padding_side = 'left'

In [11]:
rmt = RMTEncoderForSequenceClassification.from_pretrained(model_name, num_labels=3)

rmt.set_params(
                drop_empty_segments=True,
                sum_loss=False,
                input_size=None, 
                input_seg_size=None, 
                backbone_cls=None,
                inter_layer_memory=False,
                segment_ordering='repeat_first',
                num_mem_tokens=0, 
                bptt_depth=-1, 
                tokenizer=tokenizer)

### load dataset 

In [12]:
input_seq_len = 512
target_seq_len = 2

In [13]:
encode_plus_kwargs = {'max_length': input_seq_len,
                              'truncation': True,
                              'padding': 'longest',
                              'pad_to_multiple_of': 64}
generate_kwargs = {}
labels_map = {'Contradiction': 0, 'Entailment': 1, 'Not mentioned': 2}
num_labels = len(labels_map)

def collate_fn(batch):
    # cut too long strings because they may slow down tokenization
    inputs = [b['input'][:input_seq_len * 10] for b in batch]
    labels = [b['output'][:target_seq_len * 10] for b in batch]
    features = tokenizer.batch_encode_plus(list(inputs), return_tensors='pt', **encode_plus_kwargs)
    labels = np.array([labels_map[t] for t in labels])
    features['labels'] = torch.from_numpy(labels)
    return features

In [14]:
import datasets
dataset = datasets.load_dataset('tau/scrolls', 'contract_nli')
train_dataset = dataset['train']

/cephfs/home/bulatov/bulatov/hvdenv/lib/python3.8/site-packages/cryptography/hazmat/backends/openssl/x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(
Reusing dataset scrolls (/home/bulatov/.cache/huggingface/datasets/tau___scrolls/contract_nli/1.0.0/672021d5d8e1edff998a6ea7a5bff35fdfd0ae243e7cf6a8c88a57a04afb46ac)


  0%|          | 0/3 [00:00<?, ?it/s]

In [15]:
train_sampler = RandomSampler(train_dataset,)
kwargs = {'pin_memory': True, 'num_workers': 0}
train_dataloader = DataLoader(train_dataset, batch_size=3, sampler=train_sampler,
                                collate_fn=collate_fn, **kwargs)

In [262]:
gen = iter(train_dataloader)
sample = next(gen)

In [17]:
kwargs = sample.copy()
self = rmt
# kwargs
input_ids = kwargs.pop('input_ids')

memory = self.set_memory()
segmented = self.pad_and_segment(input_ids)

outputs = []
for seg_num, segment_data in enumerate(zip(*segmented)):
    input_ids, attention_mask, token_type_ids = segment_data
    if memory.ndim == 2:
        memory = memory.repeat(input_ids.shape[0], 1, 1)
    if (self.bptt_depth > -1) and (len(segmented) - seg_num > self.bptt_depth): 
        memory = memory.detach()

    seg_kwargs = dict(**kwargs)
    if self.drop_empty_segments:

        non_empty_mask = [not torch.equal(input_ids[i], self.empty) for i in range(len(input_ids))]
        if sum(non_empty_mask) == 0:
            continue
        input_ids = input_ids[non_empty_mask]
        attention_mask = attention_mask[non_empty_mask]
        token_type_ids = token_type_ids[non_empty_mask]
        seg_kwargs['labels'] = seg_kwargs['labels'][non_empty_mask]

        inputs_embeds = self.base_model.embeddings.word_embeddings(input_ids)
        inputs_embeds[:, 1:1+self.num_mem_tokens] = memory[non_empty_mask]
    else:
        inputs_embeds = self.base_model.embeddings.word_embeddings(input_ids)
        inputs_embeds[:, 1:1+self.num_mem_tokens] = memory

    seg_kwargs['inputs_embeds'] = inputs_embeds
    seg_kwargs['attention_mask'] = attention_mask
    seg_kwargs['token_type_ids'] = token_type_ids
    
    out = self.model.forward(**seg_kwargs, output_hidden_states=True, return_dict=True, )
    outputs.append(out)

    if self.drop_empty_segments:
        memory[non_empty_mask] = out.hidden_states[-1][:, :self.num_mem_tokens]
    else:
        memory = out.hidden_states[-1][:, :self.num_mem_tokens]

if self.sum_loss:
    out['loss'] = torch.stack([o['loss'] for o in outputs]).sum(dim=-1)

In [78]:
kwargs['labels']

tensor([2, 1, 2])

In [79]:
[o.logits for o in outputs]

[tensor([[-0.3037,  0.2728, -0.1070],
         [-0.3630,  0.3620, -0.0418],
         [-0.3037,  0.2728, -0.1070]], grad_fn=<AddmmBackward>),
 tensor([[-0.1574, -0.0191,  0.0029],
         [-0.1814,  0.0206, -0.0564],
         [-0.2045,  0.0097, -0.1661]], grad_fn=<AddmmBackward>)]

In [80]:
stacked_logits = torch.stack([o.logits for o in outputs])

In [82]:
probs

tensor([[[0.2502, 0.4453, 0.3046],
         [0.2250, 0.4646, 0.3103],
         [0.2502, 0.4453, 0.3046]],

        [[0.3010, 0.3457, 0.3533],
         [0.2979, 0.3646, 0.3375],
         [0.3051, 0.3779, 0.3170]]], grad_fn=<SoftmaxBackward>)

In [98]:
# mean logits
averaged_logits = stacked_logits.mean(dim=0)
probs = F.softmax(averaged_logits, dim=1)
preds = probs.argmax(dim=1)
preds

tensor([1, 1, 1])

In [175]:
# majority vote
probs = F.softmax(stacked_logits, dim=2)
preds = probs.argmax(dim=2).mode(dim=0).values
preds

tensor([1, 1, 1])

In [154]:
# most confident vote
probs = F.softmax(stacked_logits, dim=2)
most_confident_inds = probs.max(dim=2).values.argmax(dim=0)
new_probs = torch.stack([probs[ind, i] for i, ind in enumerate(most_confident_inds)])
preds = new_probs.argmax(dim=1)
preds

tensor([1, 1, 1])

In [174]:
probs

tensor([[[0.2502, 0.4453, 0.3046],
         [0.2250, 0.4646, 0.3103],
         [0.2502, 0.4453, 0.3046]],

        [[0.3010, 0.3457, 0.3533],
         [0.2979, 0.3646, 0.3375],
         [0.3051, 0.3779, 0.3170]]], grad_fn=<SoftmaxBackward>)

In [173]:
probs = F.softmax(stacked_logits, dim=2)
most_sure_seg_inds = probs.max(dim=2).values.argmax(dim=0)
# new_probs = torch.stack([probs[ind, i] for i, ind in enumerate(most_sure_seg_inds)])
# preds = new_probs.argmax(dim=1)


In [146]:
probs

tensor([[[0.2502, 0.4453, 0.3046],
         [0.2250, 0.4646, 0.3103],
         [0.2502, 0.4453, 0.3046]],

        [[0.3010, 0.3457, 0.3533],
         [0.2979, 0.3646, 0.3375],
         [0.3051, 0.3779, 0.3170]]], grad_fn=<SoftmaxBackward>)

In [150]:
most_sure_seg_inds

tensor([0, 0, 0])

In [152]:
probs[[0]]

tensor([[[0.2502, 0.4453, 0.3046],
         [0.2250, 0.4646, 0.3103],
         [0.2502, 0.4453, 0.3046]]], grad_fn=<IndexBackward>)

In [ ]:
loss = None
if labels is not None:
    if self.config.problem_type is None:
        if self.num_labels == 1:
            self.config.problem_type = "regression"
        elif self.num_labels > 1 and (labels.dtype == torch.long or labels.dtype == torch.int):
            self.config.problem_type = "single_label_classification"
        else:
            self.config.problem_type = "multi_label_classification"

    if self.config.problem_type == "regression":
        loss_fct = MSELoss()
        if self.num_labels == 1:
            loss = loss_fct(logits.squeeze(), labels.squeeze())
        else:
            loss = loss_fct(logits, labels)
    elif self.config.problem_type == "single_label_classification":
        loss_fct = CrossEntropyLoss()
        loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
    elif self.config.problem_type == "multi_label_classification":
        loss_fct = BCEWithLogitsLoss()
        loss = loss_fct(logits, labels)

### Pad and segment by tokenizer -> failed

In [250]:
model_name = "t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
encoded = tokenizer.batch_encode_plus(['Cats are actually vegetables.'])
tokenizer.batch_decode(encoded.input_ids)

['Cats are actually vegetables.</s>']

In [251]:
ids = tokenizer.encode('Cats are actually vegetables.', add_special_tokens=True)
cleaned = tokenizer.decode(ids, skip_special_tokens=True)

In [253]:
tokenizer.num_special_tokens_to_add()

1

In [231]:
model_name = 't5-base'
rmt = RMTEncoderDecoderForConditionalGeneration.from_pretrained(model_name, num_labels=3)

rmt.set_params(
                drop_empty_segments=True,
                sum_loss=False,
                input_size=None, 
                input_seg_size=None, 
                backbone_cls=None,
                inter_layer_memory=False,
                segment_ordering='repeat_first',
                num_mem_tokens=0, 
                bptt_depth=-1, 
                tokenizer=tokenizer)

In [240]:
tokenizer.batch_decode(tokenizer.encode('Cats are actually vegetables.'))

['Cat', 's', 'are', 'actually', 'vegetables', '.', '</s>']

In [249]:
# model_name = "facebook/bart-base"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# encoded = tokenizer.batch_encode_plus(['Cats are actually vegetables'])
# tokenizer.batch_decode(encoded.input_ids), tokenizer.num_special_tokens_to_add()

(['<s>Cats are actually vegetables</s>'], 2)

In [264]:
self = rmt
self.padding_side = 'right'
self.input_size = 512
self.input_seg_size = 256

self.tokenizer = tokenizer
input_ids = sample.copy()['input_ids']

In [360]:
def reencode(self, ids, add_special_tokens=False):
    # print([len(i) for i in ids])
    decoded = self.tokenizer.batch_decode(ids, skip_special_tokens=False)
    # print([len(i) for i in decoded])
    reencoded = self.tokenizer.batch_encode_plus(decoded, add_special_tokens=add_special_tokens, **encode_plus_kwargs)
    return reencoded['input_ids']


In [361]:
i1 = input_ids[1]

In [362]:
i2 = reencode(self, input_ids, True)[1]

In [363]:
len(i1), len(i2)

(512, 512)

In [367]:
tokenizer.special_tokens_map

{'eos_token': '</s>',
 'unk_token': '<unk>',
 'pad_token': '<pad>',
 'additional_special_tokens': ['<extra_id_0>',
  '<extra_id_1>',
  '<extra_id_2>',
  '<extra_id_3>',
  '<extra_id_4>',
  '<extra_id_5>',
  '<extra_id_6>',
  '<extra_id_7>',
  '<extra_id_8>',
  '<extra_id_9>',
  '<extra_id_10>',
  '<extra_id_11>',
  '<extra_id_12>',
  '<extra_id_13>',
  '<extra_id_14>',
  '<extra_id_15>',
  '<extra_id_16>',
  '<extra_id_17>',
  '<extra_id_18>',
  '<extra_id_19>',
  '<extra_id_20>',
  '<extra_id_21>',
  '<extra_id_22>',
  '<extra_id_23>',
  '<extra_id_24>',
  '<extra_id_25>',
  '<extra_id_26>',
  '<extra_id_27>',
  '<extra_id_28>',
  '<extra_id_29>',
  '<extra_id_30>',
  '<extra_id_31>',
  '<extra_id_32>',
  '<extra_id_33>',
  '<extra_id_34>',
  '<extra_id_35>',
  '<extra_id_36>',
  '<extra_id_37>',
  '<extra_id_38>',
  '<extra_id_39>',
  '<extra_id_40>',
  '<extra_id_41>',
  '<extra_id_42>',
  '<extra_id_43>',
  '<extra_id_44>',
  '<extra_id_45>',
  '<extra_id_46>',
  '<extra_id_47>',
 

In [366]:
tokenizer.decode([3, 5])

'.'

In [365]:
for i, (a, b) in enumerate(zip(i1, i2)):
    if a != b:
        print(i, a, b)

44 tensor(3) 5
46 tensor(5) 19794
47 tensor(19794) 22354
51 tensor(22354) 117
52 tensor(117) 3430
53 tensor(3430) 5
54 tensor(3) 19794
55 tensor(5) 22354
56 tensor(19794) 22354
58 tensor(22354) 6
59 tensor(22354) 1958
60 tensor(6) 13571
61 tensor(1958) 7720
62 tensor(13571) 10
63 tensor(7720) 5
64 tensor(10) 22354
65 tensor(3) 22354
66 tensor(5) 22354
67 tensor(22354) 6018
68 tensor(22354) 120
69 tensor(22354) 3
70 tensor(6018) 4822
71 tensor(120) 12
72 tensor(3) 38
73 tensor(4822) 8
74 tensor(12) 96
75 tensor(38) 13725
76 tensor(8) 725
77 tensor(96) 1280
78 tensor(13725) 1300
79 tensor(725) 7333
80 tensor(1280) 2748
81 tensor(1300) 15
82 tensor(7333) 3
83 tensor(2748) 11039
84 tensor(15) 37
85 tensor(3) 2251
86 tensor(11039) 8286
87 tensor(37) 12
88 tensor(2251) 370
89 tensor(8286) 284
90 tensor(12) 119
91 tensor(370) 28
92 tensor(284) 824
93 tensor(119) 251
94 tensor(28) 13
95 tensor(824) 8
96 tensor(251) 3669
97 tensor(13) 10261
98 tensor(8) 7
99 tensor(3669) 16
100 tensor(10261) 28

In [341]:
[len(i) for i in reencode(self, input_ids, True)]

[512, 512, 512]
[2029, 2626, 2293]


[512, 509, 512]

In [338]:
# tokenizer.special_tokens_map_extended
# tokenizer.pad_token_id

In [328]:
tokenizer.unk_token_id in input_ids[1], tokenizer.pad_token_id in input_ids[1], tokenizer.eos_token_id in input_ids[1]

(False, False, True)

In [309]:
input_ids[1][:10]

tensor([31665,    53,  3450,  1522,    59, 14765,     8,   685,    24,  7139])

In [296]:
input_ids[:1].shape, len(reencode(self, input_ids)[2])

(torch.Size([1, 512]), 511)

In [ ]:
sequence_len = input_ids.shape[1]
input_seg_size = self.input_size - self.num_mem_tokens - self.tokenizer.num_special_tokens_to_add
if self.input_seg_size is not None and self.input_seg_size < input_seg_size:
    input_seg_size = self.input_seg_size

n_segments = math.ceil(sequence_len / input_seg_size)

augmented_inputs = []
for input in input_ids:
    input = input[input != self.pad_token_id][1:-1]

    if self.padding_side == 'left':
        seg_sep_inds = [0] + list(range(len(input), 0, -input_seg_size))[::-1] # chunk so that first segment has various size
    else:
        seg_sep_inds = list(range(0, len(input), input_seg_size)) + [len(input)]
    input_segments = [input[s:e] for s, e in zip(seg_sep_inds, seg_sep_inds[1:])]

    def pad_add_special_tokens(tensor, seg_size):
        tensor = torch.cat([
                            self.mem_token_ids.to(device=self.device),
                            tensor.to(device=self.device),
                            self.eos_token.to(device=self.device)
                            ])
        pad_size = seg_size - tensor.shape[0]
        if pad_size > 0:
            tensor = F.pad(tensor, (0, pad_size))
        return tensor

    input_segments = [pad_add_special_tokens(t, self.input_size) for t in input_segments]
    empty = torch.Tensor([]).int()
    self.empty = pad_add_special_tokens(empty, self.input_size)
    empty_segments = [self.empty for i in range(n_segments - len(input_segments))]
    input_segments = empty_segments + input_segments

    augmented_input = torch.cat(input_segments)
    augmented_inputs.append(augmented_input)

augmented_inputs = torch.stack(augmented_inputs)
attention_mask = torch.ones_like(augmented_inputs)
attention_mask[augmented_inputs == self.pad_token_id] = 0

token_type_ids = torch.zeros_like(attention_mask)

input_segments = torch.chunk(augmented_inputs, n_segments, dim=1)
attention_mask = torch.chunk(attention_mask, n_segments, dim=1)
token_type_ids = torch.chunk(token_type_ids, n_segments, dim=1)

In [235]:
# sequence_len = input_ids.shape[1]
# input_seg_size = self.input_size - self.num_mem_tokens - self.tokenizer.num_special_tokens_to_add
# if self.input_seg_size is not None and self.input_seg_size < input_seg_size:
#     input_seg_size = self.input_seg_size

# n_segments = math.ceil(sequence_len / input_seg_size)

# augmented_inputs = []
# for input in input_ids:
#     input = input[input != self.pad_token_id][1:-1]

#     if self.padding_side == 'left':
#         seg_sep_inds = [0] + list(range(len(input), 0, -input_seg_size))[::-1] # chunk so that first segment has various size
#     else:
#         seg_sep_inds = list(range(0, len(input), input_seg_size)) + [len(input)]
#     input_segments = [input[s:e] for s, e in zip(seg_sep_inds, seg_sep_inds[1:])]

#     def pad_add_special_tokens(tensor, seg_size):
#         tensor = torch.cat([
#                             self.mem_token_ids.to(device=self.device),
#                             tensor.to(device=self.device),
#                             self.eos_token.to(device=self.device)
#                             ])
#         pad_size = seg_size - tensor.shape[0]
#         if pad_size > 0:
#             tensor = F.pad(tensor, (0, pad_size))
#         return tensor

#     input_segments = [pad_add_special_tokens(t, self.input_size) for t in input_segments]
#     empty = torch.Tensor([]).int()
#     self.empty = pad_add_special_tokens(empty, self.input_size)
#     empty_segments = [self.empty for i in range(n_segments - len(input_segments))]
#     input_segments = empty_segments + input_segments

#     augmented_input = torch.cat(input_segments)
#     augmented_inputs.append(augmented_input)

# augmented_inputs = torch.stack(augmented_inputs)
# attention_mask = torch.ones_like(augmented_inputs)
# attention_mask[augmented_inputs == self.pad_token_id] = 0

# token_type_ids = torch.zeros_like(attention_mask)

# input_segments = torch.chunk(augmented_inputs, n_segments, dim=1)
# attention_mask = torch.chunk(attention_mask, n_segments, dim=1)
# token_type_ids = torch.chunk(token_type_ids, n_segments, dim=1)

In [236]:
[i.shape for i in input_segments]

[torch.Size([3, 512]), torch.Size([3, 512])]

In [71]:
input_segments[0][0].shape

torch.Size([])

# Archive


## Override forward

In [24]:

import types
self = rmt

memory_storage = {'num_mem_tokens': 10}
self.base_model.encoder.forward = types.MethodType(lambda *args, **kwargs: _forward(*args, **kwargs, memory_storage=memory_storage), self.base_model.encoder)

In [26]:
sample

{'input_ids': tensor([[  101,  4909,  2283,  ...,  1017,  1012,   102],
         [  101,  2070, 14422,  ...,  2025,  3024,   102]]),
 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1]]),
 'labels': tensor([1, 2])}

In [27]:
seg_kwargs

{'pin_memory': True, 'num_workers': 0}

In [29]:
kwargs = sample.copy()
# kwargs

In [34]:
kwargs = sample.copy()
# kwargs
input_ids = kwargs.pop('input_ids')

memory = self.set_memory()
segmented = self.pad_and_segment(input_ids)

outputs = []
for seg_num, segment_data in enumerate(zip(*segmented)):
    input_ids, attention_mask, token_type_ids = segment_data
    if memory.ndim == 2:
        memory = memory.repeat(input_ids.shape[0], 1, 1)
    if (self.bptt_depth > -1) and (len(segmented) - seg_num > self.bptt_depth): 
        memory = memory.detach()

    seg_kwargs = dict(**kwargs)
    if self.drop_empty_segments:

        non_empty_mask = [not torch.equal(input_ids[i], self.empty) for i in range(len(input_ids))]
        if sum(non_empty_mask) == 0:
            continue
        input_ids = input_ids[non_empty_mask]
        attention_mask = attention_mask[non_empty_mask]
        token_type_ids = token_type_ids[non_empty_mask]
        seg_kwargs['labels'] = seg_kwargs['labels'][non_empty_mask]

        inputs_embeds = self.base_model.embeddings.word_embeddings(input_ids)
        inputs_embeds[:, 1:1+self.num_mem_tokens] = memory[non_empty_mask]
    else:
        inputs_embeds = self.base_model.embeddings.word_embeddings(input_ids)
        inputs_embeds[:, 1:1+self.num_mem_tokens] = memory

    seg_kwargs['inputs_embeds'] = inputs_embeds
    seg_kwargs['attention_mask'] = attention_mask
    seg_kwargs['token_type_ids'] = token_type_ids
    
    out = self.model.forward(**seg_kwargs, output_hidden_states=True)
    outputs.append(out)

    if self.drop_empty_segments:
        memory[non_empty_mask] = out.hidden_states[-1][:, :self.num_mem_tokens]
    else:
        memory = out.hidden_states[-1][:, :self.num_mem_tokens]

if self.sum_loss:
    out['loss'] = torch.stack([o['loss'] for o in outputs]).sum(dim=-1)

hidden states shape: (2, torch.Size([512, 256]))
 memory storage:dict_keys(['num_mem_tokens'])
hidden states shape: (2, torch.Size([512, 256]))
 memory storage:dict_keys(['num_mem_tokens', 0])
hidden states shape: (2, torch.Size([512, 256]))
 memory storage:dict_keys(['num_mem_tokens', 0, 1])
hidden states shape: (2, torch.Size([512, 256]))
 memory storage:dict_keys(['num_mem_tokens', 0, 1, 2])
hidden states shape: (2, torch.Size([512, 256]))
 memory storage:dict_keys(['num_mem_tokens', 0, 1, 2, 3])
replacing ms[i] tensor([ 0.4910, -0.3252,  0.6379,  0.4366,  0.9436,  0.3721, -0.4503,  0.2484,
        -5.6303,  0.0416], grad_fn=<SliceBackward>)... to tensor([ 0.5875, -0.4431,  0.4703,  0.1823,  0.2762, -0.0980, -0.7847, -0.3769,
        -4.0026, -0.0786], grad_fn=<SliceBackward>)
hidden states shape: (2, torch.Size([512, 256]))
 memory storage:dict_keys(['num_mem_tokens', 0, 1, 2, 3])
replacing ms[i] tensor([ 0.4131, -0.2734,  0.5131,  0.4519,  0.8615, -0.4164, -0.4695,  0.1083,
      

In [25]:
# out = self.model.forward(**seg_kwargs, output_hidden_states=True)
# out

In [ ]:
# out['hidden_states']

In [ ]:
out = forward(self.model, **seg_kwargs, output_hidden_states=True, use_cache=True)
out.keys()

odict_keys(['loss', 'logits', 'hidden_states'])

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [73]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(per_device_train_batch_size=3, per_device_eval_batch_size=1, output_dir="test_trainer", evaluation_strategy="epoch", no_cuda=True, max_steps=5)

In [74]:
# augmented_inputs = np.load('augmented_inputs.npy', allow_pickle=True)
# attn_masks = np.load('attention_masks.npy', allow_pickle=True)
# tokenizer.decode(augmented_inputs[0][512:])

In [58]:
# trainer = Trainer(
#     model=rmt,
#     # model=classic_bert,
#     args=training_args,
#     train_dataset=small_train_dataset,
#     eval_dataset=small_eval_dataset,
#     compute_metrics=compute_metrics,
# )
# trainer.train()

In [ ]:
sampler = SequentialSampler(small_train_dataset)
dl = DataLoader(small_train_dataset, sampler=sampler, 
                h_size=4)
gen = dl.__iter__()
s = next(gen)